In [1]:
import pandas as pd
import numpy as np

In [2]:
#Codename Quebec
#Reading in the total climate data file
cdtotal2 = pd.read_csv("cdtotal2.csv")

C:\Users\ROHAN\AppData\Local\Temp\ipykernel_18740\4052103858.py:3: DtypeWarning: Columns (0,4,13,19,26) have mixed types. Specify dtype option on import or set low_memory=False.
  cdtotal2 = pd.read_csv("cdtotal2.csv")


In [3]:
#Limiting decimal points to 4
cdtotal2 = round(cdtotal2, 4)

In [4]:
#Codename Quebec
#Reading in the fire + weather station minimum distance file
fire1qc = pd.read_csv("quebecfire_mindist1.csv")

In [5]:
#Limiting decimal points to 4
fire1qc = round(fire1qc, 4)

In [18]:
#Checking no. of rows and columns
fire1qc.shape

(5514, 22)

In [7]:
#Codename Quebec 007
#Since unsure of datetime format and getting object as the column datatype (strings, i guess), convert them all to 
#one particular format
#Converting "date" column from object to datetime datatype. Since "date" was not in format of mm/dd/yy and it was an object(string or list, maybe), 
#it was giving error and we had to specify that in our case, the date was in the format of dayfirst and thus had to be parsed in that way 
fire1qc["date"] = pd.to_datetime(fire1qc["date"], dayfirst = True)

#Change the format to one we like
fire1qc["firedate_formatted"] = fire1qc["date"].dt.strftime("%d/%m/%Y")

## Creating a "lightning- only", "non-zero" fire dataset

In [8]:
#only keeping the rows that has cause = L
lightning_only = fire1qc["cause"].isin(["L"])
fire2qc = fire1qc[lightning_only]

In [32]:
fire2qc.shape

(1411, 22)

In [9]:
#Using the .query method to filter for all the rows where sizeha > 0
fire3qc = fire2qc.query("sizeha > 0")

In [34]:
fire3qc.shape

(1071, 22)

In [10]:
#Changing formate of date to match that of fire1qc
cdtotal2["local_date"] = pd.to_datetime(cdtotal2["local_date"], dayfirst = True)
cdtotal2["climdate_formatted"] = cdtotal2["local_date"].dt.strftime("%d/%m/%Y")

In [25]:
#Doing this left join we had the entire fire dataset and the common pairs between fire and weather station dataset
firemerge1 = fire3qc.merge(cdtotal2, left_on = ["firedate_formatted", "climlatitude", "climlongitude"],
                           right_on = ["climdate_formatted", "y", "x"], how = "left", indicator = True)

In [12]:
#Now we will only keep the rows where indicator = both, i.e. both the fire dataset and the weather station datasets have
#common values for the above mentioned columns. Same result as the codes marked qcprevious
fire_wsmatch1 = firemerge1.query("_merge == 'both'")

In [44]:
fire_wsmatch1.shape

(840, 60)

In [39]:
firemerge1.to_csv(r"C:\\Users\\ROHAN\\erdos\\firemerge1.csv")

In [35]:
#qcprevious
#Creating a filter using Boolean indexing to select rows which share common dates between cdtotal2 and fire1qc
cdfirecom1 = cdtotal2["climdate_formatted"].isin(fire3qc["firedate_formatted"])

In [28]:
#qcprevious
#Applying filter to new dataset
cdtotal_firedate1 = cdtotal2[cdfirecom1]

In [29]:
#qcprevious
cdtotal_firedate1.shape

(73494, 37)

In [36]:
#qcprevious
#Doing a merge
fire_clim1 = pd.merge(fire3qc, cdtotal_firedate1, left_on = ["firedate_formatted", "climlatitude", "climlongitude"], 
                      right_on = ["climdate_formatted", "y", "x"], how = "inner")

In [37]:
#qcprevious
fire_clim1.shape

(840, 59)

## Creating 10 days to fire dataset

### Step 1

In [11]:
#Merging firedata to total climate data. Step 1 in creating the 10 days before the fire dataset
together2 = pd.merge(cdtotal2, fire3qc, left_on = ["climdate_formatted", "y", "x"], 
                     right_on = ["firedate_formatted", "climlatitude", "climlongitude"], how = "left", indicator = True )

In [12]:
together2.to_csv(r"C:\\Users\\ROHAN\\erdos\\together2.csv")

In [12]:
#Replace NaN with "nodate" in firedate_formatted for ease in operations
together2["firedate_formatted"] = together2["firedate_formatted"].fillna("nodate")

In [28]:
together2.head()

,climate_identifier,cooling_degree_days,cooling_degree_days_flag,direction_max_gust,direction_max_gust_flag,heating_degree_days,heating_degree_days_flag,id,local_date,local_day,...,coordinates1,climlatitude,climlongitude,firelat_radians,firelong_radians,climlat_radians,climlong_radians,distance_km,firedate_formatted,_merge
0,2200100,1.2,NaN,NaN,NaN,0.0,NaN,2200100.2012.8.2,2012-08-02,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
1,2200100,0.0,NaN,NaN,NaN,1.8,NaN,2200100.2012.8.4,2012-08-04,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
2,2200100,0.0,NaN,NaN,NaN,1.7,NaN,2200100.2012.8.5,2012-08-05,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
3,2200100,0.0,NaN,NaN,NaN,0.7,NaN,2200100.2012.8.6,2012-08-06,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
4,2200100,0.0,NaN,NaN,NaN,4.2,NaN,2200100.2012.8.7,2012-08-07,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only


### Step 2

In [29]:
#Step 2 in creating the 10 days before the fire dataset
#Get a Series of indexes that satisfy the condition I want, i.e. _merge = both
#Here we select the rows which contain the fire incidence as the together dataset is a mix of fire + nofire
cond_idx = together2[together2.loc[:, "_merge"] == "both"].index

In [28]:
cond_idx

Index([  1521,   1555,   1887,   1888,   2258,   2259,   2280,   2676,   3369,
         3370,
       ...
       730592, 731275, 731276, 732125, 732440, 732441, 741918, 744087, 744098,
       744747],
      dtype='int64', length=840)

### Step 3

In [30]:
#Step 3 in creating the 10 days before the fire dataset
#iterate through cond_idx and use .loc on dataframe to get the slices you want
dtf1 = []  #Create an empty list to store the output of your for loop
for idx in cond_idx:
    result = together2.loc[idx - 10: idx]  #For each value in cond_idx, add the 10 rows above it. total 11 rows (including the fire incidence row)
    dtf1.append(result)                    #Add successive result to the list  
out = pd.concat(dtf1, ignore_index = True) #Concatenate the final list
#Problem: Some fire incidences (fabove) had other fires above them. As each fire was selected and iterated, fabove fires had the previous fires above them
#too in their block repeated. Check 10daystofire1.csv file to understand the problem better

In [59]:
#Drop duplicates. did not work
out1 = out.drop_duplicates(subset = "coordinates", keep = "first")

In [52]:
type(out)

pandas.core.frame.DataFrame

In [61]:
out.head(3)

,climate_identifier,cooling_degree_days,cooling_degree_days_flag,direction_max_gust,direction_max_gust_flag,heating_degree_days,heating_degree_days_flag,id,local_date,local_day,...,coordinates1,climlatitude,climlongitude,firelat_radians,firelong_radians,climlat_radians,climlong_radians,distance_km,firedate_formatted,_merge
0,7060826,0.0,NaN,NaN,M,7.0,NaN,7060826.2011.5.29,2011-05-29,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
1,7060826,0.0,NaN,NaN,M,9.6,NaN,7060826.2011.5.30,2011-05-30,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
2,7060826,0.0,NaN,NaN,M,10.1,NaN,7060826.2011.5.31,2011-05-31,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only


In [55]:
out1.shape

(803, 60)

In [43]:
#Before dropping duplicates
out.to_csv(r"C:\\Users\\ROHAN\\erdos\\10daystofire1.csv")

In [56]:
#After dropping duplicates
out1.to_csv(r"C:\\Users\\ROHAN\\erdos\\10daystofire2.csv")

### Step 4

In [31]:
#Step 4 in creating the 10 days before the fire dataset
#Fill NaN column with natural numbers beginning in order
#This was done bcoz the drop_duplicate method used previously also removed the NaN when subset = coordinates. We want to keep those rows. The numbers give them the non-duplicate characteristic
m1 = out["coordinates"].isna()
out.loc[m1, "coordinates"] = m1.cumsum()

### Step 5

In [32]:
#Step 5 in creating the 10 days before the fire dataset
#Drop duplicates
out2 = out.drop_duplicates(subset = "coordinates", keep = "first")
#Problem: Because we chose 10 rows before and not 10 days/dates before, after duplicate removal only 5-6 days present in some cases instead of 10
#This happened because some fire incidences had other fire incidences above them

## 10 days to fire dataset created

In [69]:
out2.to_csv(r"C:\\Users\\ROHAN\\erdos\\10daystofire3.csv")

In [5]:
fire3 = pd.read_csv("10daystofire3.csv")

In [33]:
together2.shape

(745301, 60)

In [12]:
together2.head()

,climate_identifier,cooling_degree_days,cooling_degree_days_flag,direction_max_gust,direction_max_gust_flag,heating_degree_days,heating_degree_days_flag,id,local_date,local_day,...,coordinates1,climlatitude,climlongitude,firelat_radians,firelong_radians,climlat_radians,climlong_radians,distance_km,firedate_formatted,_merge
0,2200100,1.2,NaN,NaN,NaN,0.0,NaN,2200100.2012.8.2,2012-08-02,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
1,2200100,0.0,NaN,NaN,NaN,1.8,NaN,2200100.2012.8.4,2012-08-04,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
2,2200100,0.0,NaN,NaN,NaN,1.7,NaN,2200100.2012.8.5,2012-08-05,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
3,2200100,0.0,NaN,NaN,NaN,0.7,NaN,2200100.2012.8.6,2012-08-06,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only
4,2200100,0.0,NaN,NaN,NaN,4.2,NaN,2200100.2012.8.7,2012-08-07,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nodate,left_only


### Creating a 15day dataset using an alternative method I came up with

In [ ]:
#Codename Quebec
#Making a deep copy of tg1. tg1 is just the together2.csv file
tg3 = tg1.copy(deep = True)

In [ ]:
#Codename Quebec
#backfilled with upcoming value and front filled with next value
tg3["firedate_formattedrep"] = tg3["firedate_formatted"].bfill().ffill()

In [ ]:
#Codename Quebec
#Saving the difference in a new column
tg3["difference"] = (tg3["firedate_formattedrep"] - tg3["climdate_formatted"]).dt.days

In [ ]:
#Codename Quebec
tg4 = tg3.query("0 <= difference <= 15")

In [ ]:
tg4.to_csv(r"C:\\Users\\ROHAN\\erdos\\15daystofire1.csv")

###Kenny's method

In [25]:
(
    together2
    .query("climate_identifier == 7060826")
    .filter([
        "climdate_formatted", "firedate_formatted", 
        "latitude", "longitude", "y", "x", "fid"
    ])
    .assign(maxfiredate = lambda df)
)

array(['nodate', '08/06/2011', '12/07/2011', '08/06/2012', '13/06/2013',
       '05/07/2013', '11/08/2014', '10/07/2016', '23/06/2017',
       '04/07/2017', '28/06/2020'], dtype=object)

In [23]:
together2.columns

Index(['climate_identifier', 'cooling_degree_days', 'cooling_degree_days_flag',
       'direction_max_gust', 'direction_max_gust_flag', 'heating_degree_days',
       'heating_degree_days_flag', 'id', 'local_date', 'local_day',
       'local_month', 'local_year', 'max_rel_humidity',
       'max_rel_humidity_flag', 'max_temperature', 'max_temperature_flag',
       'mean_temperature', 'mean_temperature_flag', 'min_rel_humidity',
       'min_rel_humidity_flag', 'min_temperature', 'min_temperature_flag',
       'province_code', 'snow_on_ground', 'snow_on_ground_flag',
       'speed_max_gust', 'speed_max_gust_flag', 'station_name',
       'total_precipitation', 'total_precipitation_flag', 'total_rain',
       'total_rain_flag', 'total_snow', 'total_snow_flag', 'x', 'y',
       'climdate_formatted', 'oldidentifierfrompreviousfile', 'fid',
       'src_agency', 'latitude', 'longitude', 'date', 'sizeha', 'cause',
       'protzone', 'ecoz_name', 'wildlat1', 'coordinates', 'Unnamed: 0_y',
       '

In [ ]:
together2.query("climate_identifier")

In [14]:
fire_df = together2.query('_merge == "both"')

In [15]:
fire_df

,climate_identifier,cooling_degree_days,cooling_degree_days_flag,direction_max_gust,direction_max_gust_flag,heating_degree_days,heating_degree_days_flag,id,local_date,local_day,...,coordinates1,climlatitude,climlongitude,firelat_radians,firelong_radians,climlat_radians,climlong_radians,distance_km,firedate_formatted,_merge
1521,7060826,0.0,E,NaN,M,6.4,E,7060826.2011.6.8,2011-06-08,8,...,"50.7289,-71.0131",50.7289,-71.0131,0.8814,-1.2536,0.8854,-1.2394,62.6493,08/06/2011,both
1555,7060826,0.0,NaN,NaN,M,3.9,NaN,7060826.2011.7.12,2011-07-12,12,...,"50.7289,-71.0131",50.7289,-71.0131,0.8887,-1.2474,0.8854,-1.2394,38.2989,12/07/2011,both
1887,7060826,0.0,NaN,NaN,M,3.9,NaN,7060826.2012.6.8,2012-06-08,8,...,"50.7289,-71.0131",50.7289,-71.0131,0.8852,-1.2560,0.8854,-1.2394,66.7388,08/06/2012,both
1888,7060826,0.0,NaN,NaN,M,3.9,NaN,7060826.2012.6.8,2012-06-08,8,...,"50.7289,-71.0131",50.7289,-71.0131,0.8847,-1.2558,0.8854,-1.2394,66.2250,08/06/2012,both
2258,7060826,0.0,NaN,0.0,NaN,1.5,NaN,7060826.2013.6.13,2013-06-13,13,...,"50.7289,-71.0131",50.7289,-71.0131,0.8872,-1.2486,0.8854,-1.2394,38.8093,13/06/2013,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732441,7098908,0.0,NaN,7.0,NaN,1.4,NaN,7098908.2021.6.18,2021-06-18,18,...,"53.0106,-78.8311",53.0106,-78.8311,0.9358,-1.3770,0.9252,-1.3759,67.2774,18/06/2021,both
741918,7038975,NaN,M,NaN,NaN,NaN,M,7038975.2012.7.27,2012-07-27,27,...,"46.0667,-76.05",46.0667,-76.0500,0.8050,-1.3374,0.8040,-1.3273,44.8810,27/07/2012,both
744087,7038975,NaN,NaN,NaN,NaN,NaN,NaN,7038975.2018.7.5,2018-07-05,5,...,"46.0667,-76.05",46.0667,-76.0500,0.8004,-1.3258,0.8040,-1.3273,24.1993,05/07/2018,both
744098,7038975,NaN,NaN,NaN,NaN,NaN,NaN,7038975.2018.7.16,2018-07-16,16,...,"46.0667,-76.05",46.0667,-76.0500,0.8011,-1.3249,0.8040,-1.3273,21.1434,16/07/2018,both


In [16]:
fire_df2 = fire_df.assign(max_date = lambda df: df.firedate_formatted)

In [17]:
fire_df2

,climate_identifier,cooling_degree_days,cooling_degree_days_flag,direction_max_gust,direction_max_gust_flag,heating_degree_days,heating_degree_days_flag,id,local_date,local_day,...,climlatitude,climlongitude,firelat_radians,firelong_radians,climlat_radians,climlong_radians,distance_km,firedate_formatted,_merge,max_date
1521,7060826,0.0,E,NaN,M,6.4,E,7060826.2011.6.8,2011-06-08,8,...,50.7289,-71.0131,0.8814,-1.2536,0.8854,-1.2394,62.6493,08/06/2011,both,08/06/2011
1555,7060826,0.0,NaN,NaN,M,3.9,NaN,7060826.2011.7.12,2011-07-12,12,...,50.7289,-71.0131,0.8887,-1.2474,0.8854,-1.2394,38.2989,12/07/2011,both,12/07/2011
1887,7060826,0.0,NaN,NaN,M,3.9,NaN,7060826.2012.6.8,2012-06-08,8,...,50.7289,-71.0131,0.8852,-1.2560,0.8854,-1.2394,66.7388,08/06/2012,both,08/06/2012
1888,7060826,0.0,NaN,NaN,M,3.9,NaN,7060826.2012.6.8,2012-06-08,8,...,50.7289,-71.0131,0.8847,-1.2558,0.8854,-1.2394,66.2250,08/06/2012,both,08/06/2012
2258,7060826,0.0,NaN,0.0,NaN,1.5,NaN,7060826.2013.6.13,2013-06-13,13,...,50.7289,-71.0131,0.8872,-1.2486,0.8854,-1.2394,38.8093,13/06/2013,both,13/06/2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732441,7098908,0.0,NaN,7.0,NaN,1.4,NaN,7098908.2021.6.18,2021-06-18,18,...,53.0106,-78.8311,0.9358,-1.3770,0.9252,-1.3759,67.2774,18/06/2021,both,18/06/2021
741918,7038975,NaN,M,NaN,NaN,NaN,M,7038975.2012.7.27,2012-07-27,27,...,46.0667,-76.0500,0.8050,-1.3374,0.8040,-1.3273,44.8810,27/07/2012,both,27/07/2012
744087,7038975,NaN,NaN,NaN,NaN,NaN,NaN,7038975.2018.7.5,2018-07-05,5,...,46.0667,-76.0500,0.8004,-1.3258,0.8040,-1.3273,24.1993,05/07/2018,both,05/07/2018
744098,7038975,NaN,NaN,NaN,NaN,NaN,NaN,7038975.2018.7.16,2018-07-16,16,...,46.0667,-76.0500,0.8011,-1.3249,0.8040,-1.3273,21.1434,16/07/2018,both,16/07/2018


In [ ]:
date1 = together2.query("climdate")